# Advent of Code

## 2023-012-010
## 2023 010

https://adventofcode.com/2023/day/10

In [4]:
# Parsing the input and implementing the logic to solve the maze problem
# Importing necessary libraries
import numpy as np
from collections import deque

# Read the uploaded input file
file_path = 'sample-input.txt'
with open(file_path, 'r') as f:
    input_data = f.read().splitlines()

# Initialize the grid
grid = np.array([list(line) for line in input_data])

# Define the directions for traversal
DIRECTIONS = {
    '|': [(0, -1), (0, 1)],  # vertical
    '-': [(-1, 0), (1, 0)],  # horizontal
    'L': [(0, -1), (-1, 0)],  # north and east
    'J': [(0, -1), (1, 0)],  # north and west
    '7': [(0, 1), (1, 0)],   # south and west
    'F': [(0, 1), (-1, 0)],  # south and east
}

# Locate the starting position
start_position = None
for r in range(grid.shape[0]):
    for c in range(grid.shape[1]):
        if grid[r, c] == 'S':
            start_position = (r, c)
            break
    if start_position:
        break

# BFS to find the loop and distances
def find_loop_and_distances(start, grid):
    queue = deque([(start, 0)])
    visited = set()
    distances = {}

    while queue:
        (x, y), dist = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        distances[(x, y)] = dist

        current_pipe = grid[x, y]
        if current_pipe == 'S':
            current_pipe = 'F'  # Assumption based on the problem statement

        for dx, dy in DIRECTIONS.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx, ny] != '.':
                queue.append(((nx, ny), dist + 1))

    return distances

# Find the loop and distances
distances = find_loop_and_distances(start_position, grid)

# Find the farthest point in the loop
farthest_distance = max(distances.values())

farthest_distance

7

In [14]:
import numpy as np
from collections import deque

def is_valid_position(x, y, grid):
    """Check if a position is within bounds and not ground."""
    return 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1] and grid[x, y] != '.'

def mark_loop(grid, start, directions):
    """Identify the loop in the grid and return a grid with non-loop parts replaced with '.'."""
    queue = deque([start])
    visited = set()
    loop_grid = np.full(grid.shape, '.', dtype=str)

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        loop_grid[x, y] = grid[x, y]

        current_pipe = grid[x, y]
        if current_pipe == 'S':
            current_pipe = 'F'  # Treat 'S' as a connected pipe

        for dx, dy in directions.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if is_valid_position(nx, ny, grid) and (nx, ny) not in visited:
                # Ensure connectivity matches in both directions
                reverse_dx, reverse_dy = -dx, -dy
                neighbor_pipe = grid[nx, ny]
                if neighbor_pipe == 'S':
                    neighbor_pipe = 'F'
                if (reverse_dx, reverse_dy) in directions.get(neighbor_pipe, []):
                    queue.append((nx, ny))

    return loop_grid

def loop_to_string(loop_grid):
    """Convert the loop grid to a string representation with '.' for non-loop parts."""
    return '\n'.join(''.join(row) for row in loop_grid)

def find_start(grid):
    """Locate the starting position in the grid."""
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if grid[r, c] == 'S':
                return r, c

# Define the direction mappings
DIRECTIONS = {
    '|': [(-1, 0), (1, 0)],  # vertical: north and south
    '-': [(0, -1), (0, 1)],  # horizontal: west and east
    'L': [(-1, 0), (0, 1)],  # north and east
    'J': [(-1, 0), (0, -1)],  # north and west
    '7': [(1, 0), (0, -1)],   # south and west
    'F': [(1, 0), (0, 1)],    # south and east
    'S': [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Start pipe connects in all directions
}

# Parse the files and process
def parse_and_print_loops(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()

    grid = np.array([list(row) for row in data])
    start = find_start(grid)
    loop_grid = mark_loop(grid, start, DIRECTIONS)
    print(loop_to_string(loop_grid))

# Example usage:
# For sample input
print("Sample Loop:")
parse_and_print_loops('sample-input.txt')

# For full input
print("\nFull Loop:")
parse_and_print_loops('input.txt')

Sample Loop:
..F7.
.FJ|.
SJ.L7
|F--J
LJ...

Full Loop:
.............................................................................................F----7.........................................
...............................................F7............................................L---7L7........................................
.............................................F-J|................................................|FJ........................................
.............................................|F7|...F7..........................................FJ|.........................................
...........................................F-J|LJF--J|........................................F-JFJ.........................................
...........................................L7FJ.FJF--J........................................L-7|..........................................
............................................|L--JFJF7...F7.F7.................................F-JL-

In [15]:
import numpy as np
from collections import deque

def is_valid_position(x, y, grid):
    """Check if a position is within bounds and not ground."""
    return 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1] and grid[x, y] != '.'

def compute_distances(loop_grid, start, directions):
    """Perform BFS to compute distances along the loop from the starting point."""
    queue = deque([(start, 0)])
    visited = set()
    distances = {}

    while queue:
        (x, y), dist = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        distances[(x, y)] = dist

        current_pipe = loop_grid[x, y]
        if current_pipe == '.':
            continue  # Ignore non-loop tiles

        for dx, dy in directions.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if (nx, ny) not in visited and 0 <= nx < loop_grid.shape[0] and 0 <= ny < loop_grid.shape[1]:
                if loop_grid[nx, ny] != '.':
                    queue.append(((nx, ny), dist + 1))

    return distances

def mark_loop_and_number(grid, start, directions):
    """Identify the loop and replace loop parts with the last digit of the distance."""
    loop_grid = mark_loop(grid, start, directions)  # Identify loop parts
    distances = compute_distances(loop_grid, start, directions)  # Compute distances

    numbered_grid = np.full(grid.shape, '.', dtype=str)
    for (x, y), dist in distances.items():
        numbered_grid[x, y] = str(dist % 10)  # Replace with the last digit of the distance

    return numbered_grid

def loop_to_string(loop_grid):
    """Convert the loop grid to a string representation."""
    return '\n'.join(''.join(row) for row in loop_grid)

def find_start(grid):
    """Locate the starting position in the grid."""
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if grid[r, c] == 'S':
                return r, c

# Define the direction mappings
DIRECTIONS = {
    '|': [(-1, 0), (1, 0)],  # vertical: north and south
    '-': [(0, -1), (0, 1)],  # horizontal: west and east
    'L': [(-1, 0), (0, 1)],  # north and east
    'J': [(-1, 0), (0, -1)],  # north and west
    '7': [(1, 0), (0, -1)],   # south and west
    'F': [(1, 0), (0, 1)],    # south and east
    'S': [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Start pipe connects in all directions
}

# Parse the files and process
def parse_and_number_loops(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()

    grid = np.array([list(row) for row in data])
    start = find_start(grid)
    numbered_grid = mark_loop_and_number(grid, start, DIRECTIONS)
    print(loop_to_string(numbered_grid))

# Example usage:
# For sample input
print("Sample Loop with Numbers:")
parse_and_number_loops('sample-input.txt')

# For full input
print("\nFull Loop with Numbers:")
parse_and_number_loops('input.txt')

Sample Loop with Numbers:
..45.
.236.
01.78
14567
23...

Full Loop with Numbers:
.............................................................................................321098.........................................
...............................................43............................................4567876........................................
.............................................7652................................................945........................................
.............................................8781...32..........................................103.........................................
...........................................10969076541........................................43212.........................................
...........................................2345.987890........................................5670..........................................
............................................432105676...21.54............

In [18]:
import numpy as np
from collections import deque

def is_valid_position(x, y, grid):
    """Check if a position is within bounds and not ground."""
    return 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1] and grid[x, y] != '.'

def mark_loop(grid, start, directions):
    """Identify the loop in the grid and return a grid with non-loop parts replaced with '.'."""
    queue = deque([start])
    visited = set()
    loop_grid = np.full(grid.shape, '.', dtype=str)

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        loop_grid[x, y] = grid[x, y]

        current_pipe = grid[x, y]
        if current_pipe == 'S':
            current_pipe = 'F'  # Treat 'S' as a connected pipe

        for dx, dy in directions.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if is_valid_position(nx, ny, grid) and (nx, ny) not in visited:
                # Ensure connectivity matches in both directions
                reverse_dx, reverse_dy = -dx, -dy
                neighbor_pipe = grid[nx, ny]
                if neighbor_pipe == 'S':
                    neighbor_pipe = 'F'
                if (reverse_dx, reverse_dy) in directions.get(neighbor_pipe, []):
                    queue.append((nx, ny))

    return loop_grid

def count_loop_tiles(loop_grid):
    """Count the number of loop tiles in the grid."""
    return np.sum(loop_grid != '.')  # Count all tiles that are not '.'

def find_start(grid):
    """Locate the starting position in the grid."""
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if grid[r, c] == 'S':
                return r, c

# Define the direction mappings
DIRECTIONS = {
    '|': [(-1, 0), (1, 0)],  # vertical: north and south
    '-': [(0, -1), (0, 1)],  # horizontal: west and east
    'L': [(-1, 0), (0, 1)],  # north and east
    'J': [(-1, 0), (0, -1)],  # north and west
    '7': [(1, 0), (0, -1)],   # south and west
    'F': [(1, 0), (0, 1)],    # south and east
    'S': [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Start pipe connects in all directions
}

# Parse the file and compute the maximum distance
def parse_and_compute_max_distance(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()

    grid = np.array([list(row) for row in data])
    start = find_start(grid)
    loop_grid = mark_loop(grid, start, DIRECTIONS)
    loop_tile_count = count_loop_tiles(loop_grid)
    max_distance = loop_tile_count // 2  # Maximum distance is half the loop size
    return max_distance

# Example usage:
# For sample input
print("Sample Maximum Distance:")
sample_max_distance = parse_and_compute_max_distance('sample-input.txt')
print(sample_max_distance)

# For full input
print("\nFull Maximum Distance:")
full_max_distance = parse_and_compute_max_distance('input.txt')
print(full_max_distance)

Sample Maximum Distance:
8

Full Maximum Distance:
6768


In [21]:
import numpy as np
from collections import deque

def is_valid_position(x, y, grid):
    """Check if a position is within bounds."""
    return 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1]

def mark_loop(grid, start, directions):
    """Identify the loop in the grid and return a grid with non-loop parts replaced with '.'."""
    queue = deque([start])
    visited = set()
    loop_grid = np.full(grid.shape, '.', dtype=str)

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        loop_grid[x, y] = grid[x, y]

        current_pipe = grid[x, y]
        if current_pipe == 'S':
            current_pipe = 'F'  # Treat 'S' as a connected pipe

        for dx, dy in directions.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if is_valid_position(nx, ny, grid) and (nx, ny) not in visited:
                # Ensure connectivity matches in both directions
                reverse_dx, reverse_dy = -dx, -dy
                neighbor_pipe = grid[nx, ny]
                if neighbor_pipe == 'S':
                    neighbor_pipe = 'F'
                if (reverse_dx, reverse_dy) in directions.get(neighbor_pipe, []):
                    queue.append((nx, ny))

    return loop_grid

def flood_fill(grid, start, fill_char):
    """Flood fill from the starting position to mark areas as inside or outside."""
    queue = deque([start])
    visited = set()

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited or not is_valid_position(x, y, grid) or grid[x, y] != '.':
            continue
        visited.add((x, y))
        grid[x, y] = fill_char  # Mark the tile

        # Explore all neighbors
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            queue.append((x + dx, y + dy))

def count_tiles(grid, char):
    """Count the number of tiles with the specified character."""
    return np.sum(grid == char)

def identify_inside_outside(grid, loop_grid):
    """Mark inside (I) and outside (O) regions and count inside tiles."""
    marked_grid = loop_grid.copy()

    # Flood fill from edges to mark outside ('O') tiles
    for row in range(grid.shape[0]):
        if marked_grid[row, 0] == '.':
            flood_fill(marked_grid, (row, 0), 'O')
        if marked_grid[row, grid.shape[1] - 1] == '.':
            flood_fill(marked_grid, (row, grid.shape[1] - 1), 'O')
    for col in range(grid.shape[1]):
        if marked_grid[0, col] == '.':
            flood_fill(marked_grid, (0, col), 'O')
        if marked_grid[grid.shape[0] - 1, col] == '.':
            flood_fill(marked_grid, (grid.shape[0] - 1, col), 'O')

    # Flood fill remaining '.' tiles to mark inside ('I') tiles
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            if marked_grid[row, col] == '.':
                flood_fill(marked_grid, (row, col), 'I')

    # Count the number of inside tiles
    inside_tile_count = count_tiles(marked_grid, 'I')

    return marked_grid, inside_tile_count

def find_start(grid):
    """Locate the starting position in the grid."""
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if grid[r, c] == 'S':
                return r, c

# Define the direction mappings
DIRECTIONS = {
    '|': [(-1, 0), (1, 0)],  # vertical: north and south
    '-': [(0, -1), (0, 1)],  # horizontal: west and east
    'L': [(-1, 0), (0, 1)],  # north and east
    'J': [(-1, 0), (0, -1)],  # north and west
    '7': [(1, 0), (0, -1)],   # south and west
    'F': [(1, 0), (0, 1)],    # south and east
    'S': [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Start pipe connects in all directions
}

# Parse the file and compute inside tiles
def parse_and_compute_inside_tiles(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()

    grid = np.array([list(row) for row in data])
    start = find_start(grid)
    loop_grid = mark_loop(grid, start, DIRECTIONS)
    marked_grid, inside_tile_count = identify_inside_outside(grid, loop_grid)
    return marked_grid, inside_tile_count

# Example usage:
# For sample input
print("Sample Input:")
sample_marked_grid, sample_inside_count = parse_and_compute_inside_tiles('sample-input.txt')
print('\n'.join(''.join(row) for row in sample_marked_grid))
print(f"Inside tiles: {sample_inside_count}")

# For full input
print("\nFull Input:")
full_marked_grid, full_inside_count = parse_and_compute_inside_tiles('input.txt')
print('\n'.join(''.join(row) for row in full_marked_grid))  # Limit rows for readability
print(f"Inside tiles: {full_inside_count}")

Sample Input:
OOF7O
OFJ|O
SJIL7
|F--J
LJOOO
Inside tiles: 1

Full Input:
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF----7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL---7L7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-J|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|FJOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|F7|OOOF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOFJ|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-J|LJF--J|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-JFJOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL7FJIFJF--JOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL-7|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|L--JFJF7OOOF7OF7OOOOOOOOOOOOOOOOOOOO

610 That's not the right answer; your answer is too high. If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit. Please wait one minute before trying again. [Return to Day 10]

In [23]:
import numpy as np
from collections import deque

def is_valid_position(x, y, grid):
    """Check if a position is within bounds."""
    return 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1]

def mark_loop(grid, start, directions):
    """Identify the loop in the grid and return a grid with non-loop parts replaced with '.'."""
    queue = deque([start])
    visited = set()
    loop_grid = np.full(grid.shape, '.', dtype=str)

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        loop_grid[x, y] = grid[x, y]

        current_pipe = grid[x, y]
        if current_pipe == 'S':
            current_pipe = 'F'  # Treat 'S' as a connected pipe

        for dx, dy in directions.get(current_pipe, []):
            nx, ny = x + dx, y + dy
            if is_valid_position(nx, ny, grid) and (nx, ny) not in visited:
                # Ensure connectivity matches in both directions
                reverse_dx, reverse_dy = -dx, -dy
                neighbor_pipe = grid[nx, ny]
                if neighbor_pipe == 'S':
                    neighbor_pipe = 'F'
                if (reverse_dx, reverse_dy) in directions.get(neighbor_pipe, []):
                    queue.append((nx, ny))

    return loop_grid

def flood_fill(grid, start, fill_char):
    """Flood fill from the starting position to mark areas as inside or outside."""
    queue = deque([start])
    visited = set()

    while queue:
        x, y = queue.popleft()
        if (x, y) in visited or not is_valid_position(x, y, grid) or grid[x, y] != '.':
            continue
        visited.add((x, y))
        grid[x, y] = fill_char  # Mark the tile

        # Explore all neighbors
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            queue.append((x + dx, y + dy))

def count_tiles(grid, char):
    """Count the number of tiles with the specified character."""
    return np.sum(grid == char)

def identify_inside_outside(grid, loop_grid):
    """Mark inside (I) and outside (O) regions and count inside tiles."""
    marked_grid = loop_grid.copy()

    # Flood fill from edges to mark outside ('O') tiles
    for row in range(grid.shape[0]):
        if marked_grid[row, 0] == '.':
            flood_fill(marked_grid, (row, 0), 'O')
        if marked_grid[row, grid.shape[1] - 1] == '.':
            flood_fill(marked_grid, (row, grid.shape[1] - 1), 'O')
    for col in range(grid.shape[1]):
        if marked_grid[0, col] == '.':
            flood_fill(marked_grid, (0, col), 'O')
        if marked_grid[grid.shape[0] - 1, col] == '.':
            flood_fill(marked_grid, (grid.shape[0] - 1, col), 'O')

    # Flood fill remaining '.' tiles to mark inside ('I') tiles
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            if marked_grid[row, col] == '.':
                flood_fill(marked_grid, (row, col), 'I')

    # Count the number of inside tiles
    inside_tile_count = count_tiles(marked_grid, 'I')

    return marked_grid, inside_tile_count

def loop_to_string(loop_grid):
    """Convert the loop grid to a string representation."""
    return '\n'.join(''.join(row) for row in loop_grid)

def find_start(grid):
    """Locate the starting position in the grid."""
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if grid[r, c] == 'S':
                return r, c

# Define the direction mappings
DIRECTIONS = {
    '|': [(-1, 0), (1, 0)],  # vertical: north and south
    '-': [(0, -1), (0, 1)],  # horizontal: west and east
    'L': [(-1, 0), (0, 1)],  # north and east
    'J': [(-1, 0), (0, -1)],  # north and west
    '7': [(1, 0), (0, -1)],   # south and west
    'F': [(1, 0), (0, 1)],    # south and east
    'S': [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Start pipe connects in all directions
}

# Parse the file and compute inside tiles
def parse_and_compute_inside_tiles(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()

    grid = np.array([list(row) for row in data])
    start = find_start(grid)
    loop_grid = mark_loop(grid, start, DIRECTIONS)
    marked_grid, inside_tile_count = identify_inside_outside(grid, loop_grid)
    return loop_to_string(marked_grid), inside_tile_count

# For full input
print("Full Input:")
full_marked_grid, full_inside_count = parse_and_compute_inside_tiles('input.txt')
print(full_marked_grid)  # Print the entire grid for the full input
print(f"Inside tiles: {full_inside_count}")

Full Input:
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF----7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL---7L7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-J|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|FJOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|F7|OOOF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOFJ|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-J|LJF--J|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-JFJOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL7FJIFJF--JOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOL-7|OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO|L--JFJF7OOOF7OF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-JL-7OF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
O